nn.DataParallel()只能处理模型

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 定义神经网络模型
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # 定义你的模型结构
        self.fc = nn.Linear(in_features=10, out_features=1)
    
    def forward(self, x):
        # 定义前向传播逻辑
        x = self.fc(x)
        return x

# 定义数据集
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

# 假设有10张图片数据
data = torch.randn(10, 10)  # 假设每张图片有10个特征

# 创建数据集和数据加载器
dataset = MyDataset(data)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# 初始化模型并放到GPU上
model = MyModel().cuda()

# 将模型在GPU上进行并行化
model = nn.DataParallel(model)


In [12]:
x = 0
for batch in dataloader:
  # 将数据移动到GPU上
  inputs = batch.cuda()
  
  # 正向传播
  outputs = model(inputs)
  x = x+1
  print(x)
  print(outputs)

1
tensor([[0.3122]], device='cuda:0', grad_fn=<AddmmBackward0>)
2
tensor([[0.2484]], device='cuda:0', grad_fn=<AddmmBackward0>)
3
tensor([[0.3374]], device='cuda:0', grad_fn=<AddmmBackward0>)
4
tensor([[-0.1192]], device='cuda:0', grad_fn=<AddmmBackward0>)
5
tensor([[0.5996]], device='cuda:0', grad_fn=<AddmmBackward0>)
6
tensor([[0.2356]], device='cuda:0', grad_fn=<AddmmBackward0>)
7
tensor([[-0.1340]], device='cuda:0', grad_fn=<AddmmBackward0>)
8
tensor([[0.0684]], device='cuda:0', grad_fn=<AddmmBackward0>)
9
tensor([[-1.0539]], device='cuda:0', grad_fn=<AddmmBackward0>)
10
tensor([[-0.1103]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

num_epochs = 1000

# 训练模型
for epoch in range(num_epochs):
    for batch in dataloader:
        # 将数据移动到GPU上
        inputs = batch.cuda()
        
        # 正向传播
        outputs = model(inputs)
        
        # 计算损失
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


torch.multiprocessing，可以给函数使用

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

# 定义神经网络模型
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # 定义你的模型结构
        self.fc = nn.Linear(in_features=1, out_features=1)
    
    def forward(self, x):
        # 定义前向传播逻辑
        x = self.fc(x)
        return x

# 定义一个函数，用于在多进程中执行神经网络的计算
def worker_function(index, input_data):
    # 初始化模型
    model = MyModel()
    
    # 定义损失函数和优化器
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    # 准备数据
    inputs = torch.tensor([[input_data[index]]], dtype=torch.float32)
    labels = torch.tensor([[input_data[index] * 2]], dtype=torch.float32)  # 举例：输出为输入的两倍
    
    # 训练模型
    for _ in range(100):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # 返回输出结果
    return outputs.item()

if __name__ == '__main__':
    # 准备数据
    input_data = [1, 2, 3, 4, 5]
    
    # 创建进程池
    num_processes = len(input_data)
    pool = mp.Pool(processes=num_processes)
    
    # 在进程池中并行执行神经网络计算
    results = [pool.apply_async(worker_function, args=(i, input_data)) for i in range(num_processes)]
    
    # 收集所有进程的结果
    final_results = [result.get() for result in results]
    
    # 输出结果
    print("Final Results:", final_results)


Process 3: Result = tensor([ 3,  6,  9, 12, 15])Process 0: Result = tensor([0, 0, 0, 0, 0])Process 2: Result = tensor([ 2,  4,  6,  8, 10])Process 1: Result = tensor([1, 2, 3, 4, 5])





In [39]:
import torch
import numpy as np

# 假设你有一个输入张量 input_tensor
# input_tensor 的形状为 [batch_size, history_idx, cam_id, channel, width, height]

# 生成示例输入数据（假设batch_size=2，history_idx=3，cam_id=4，channel=3，width=5，height=5）
batch_size = 2
history_idx = 3
cam_id = 4
channel = 3
width = 5
height = 5

input_tensor = torch.randn(batch_size, history_idx, cam_id, channel, width, height)

print(f"{input_tensor.shape=}")

# 将输入张量重塑成 [batch_size*history_idx, cam_id, channel, width, height]
input_shape = input_tensor.shape
target_shape = np.append(-1, input_shape[2:])
print(f"{target_shape}")
print([-1, cam_id, channel, width, height])

reshaped_tensor = input_tensor.view(-1, cam_id, channel, width, height)
reshaped_tensor1 = input_tensor.view(target_shape[0], target_shape[1], target_shape[2], target_shape[3], target_shape[4])

# 恢复原来的形状
output_tensor = reshaped_tensor.view(batch_size, history_idx, cam_id, channel, width, height)

# 打印结果
print("原始形状：", input_tensor.shape)
print("重塑后的形状：", reshaped_tensor.shape)
print(input_tensor[1,2,0,0,0,2])
print(reshaped_tensor[5,0,0,0,2])
print(output_tensor[1,2,0,0,0,2])
print("重塑后的形状：", reshaped_tensor1.shape)

print("恢复后的形状：", output_tensor.shape)


input_tensor.shape=torch.Size([2, 3, 4, 3, 5, 5])
[-1  4  3  5  5]
[-1, 4, 3, 5, 5]
原始形状： torch.Size([2, 3, 4, 3, 5, 5])
重塑后的形状： torch.Size([6, 4, 3, 5, 5])
tensor(0.1753)
tensor(0.1753)
tensor(0.1753)
重塑后的形状： torch.Size([6, 4, 3, 5, 5])
恢复后的形状： torch.Size([2, 3, 4, 3, 5, 5])


In [48]:
import numpy as np
import cv2 as cv

image = np.ones([480,640,3])

image_dict = dict()
image_dict["wrist"] = image

image_list = []
for idx in range(5):
  image_list.append(image_dict)

image2 = np.ones([120,160,3])
image_dict2 = dict()
image_dict2["wrist"] = image2
image_list.append(image_dict2)

print("图片列表长度：",len(image_list)) # 妈的大问题，如果一个list又形状不同的元素，就无法使用np.shape

# 查看列表shape
# print("查看列表shape:")
# for idx in range(len(image_list)):
#   print(np.shape(image_list[idx]["wrist"]))


# for idx in range(2):
#   image_list[idx]["wrist"] = cv.resize(image_list[idx]["wrist"], (0, 0), fx=0.25, fy=0.25, interpolation=cv.INTER_LINEAR)

for idx in range(2):
  image_list[idx]["wrist"] = cv.resize(image_list[idx]["wrist"], (0, 0), fx=0.25, fy=0.25, interpolation=cv.INTER_LINEAR)

# 查看列表shape
print("查看列表shape:")
for idx in range(len(image_list)):
  print(np.shape(image_list[idx]["wrist"]))
  

图片列表长度： 6
查看列表shape:
(30, 40, 3)
(30, 40, 3)
(30, 40, 3)
(30, 40, 3)
(30, 40, 3)
(120, 160, 3)


In [49]:
import numpy as np
import cv2 as cv

image = np.ones([480,640,3])

image_dict = dict() # 又是dict的对象指针问题，操你妈
image_dict["wrist"] = image

image_list = []
for idx in range(5):
  image_list.append(image_dict.copy())  # 使用 .copy() 方法来避免列表中所有元素指向同一字典

image2 = np.ones([120,160,3])
image_dict2 = dict()
image_dict2["wrist"] = image2
image_list.append(image_dict2)

print(len(image_list))

# 统一图像大小
# for idx in range(len(image_list)):
#     image_list[idx]["wrist"] = cv.resize(image_list[idx]["wrist"], (640, 480), interpolation=cv.INTER_LINEAR)

for idx in range(2):
  image_list[idx]["wrist"] = cv.resize(image_list[idx]["wrist"], (0, 0), fx=0.25, fy=0.25, interpolation=cv.INTER_LINEAR)

for idx in range(len(image_list)):
  print(np.shape(image_list[idx]["wrist"]))


6
(120, 160, 3)
(120, 160, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(120, 160, 3)


In [7]:
import numpy as np

np.random.choice(range(15,20), size=3, replace=False)

array([18, 15, 17])

In [19]:
x = [10, 5]
# x = reversed(x)

# print(x[-1])

y = 12

for i in x:
  
  if y >= i:
    print(i)
    break
  
  elif y < i and i != x[-1]:
    continue
  
  print(i)

10


In [23]:
for i in range(10, 0, -1):
  print(i)

10
9
8
7
6
5
4
3
2
1
